In [38]:
import glob
import pandas as pd

csv_files = glob.glob("data/result/test-examples_*.csv")

df_list = [pd.read_csv(file) for file in csv_files]
concatenated_df = pd.concat(df_list, axis=0, ignore_index=True)
concatenated_df.to_excel("data/news_category_classification/Final.xlsx", index=False)

print("Concatenation complete")

Concatenation complete


In [39]:
import pandas as pd

df = pd.read_excel("data/news_category_classification/Final.xlsx")
df

,category,text,predicted_category_Llama3-8b-8192,predicted_category_Llama3-70b-8192,predicted_category_Mixtral-8x7b-32768,predicted_category_Gemma-7b-It,predicted_category_gpt-3.5-turbo
0,politics,nhs waiting time target is cut hospital waitin...,politics,others,politics_x000D_\n_x000D_\nthe text is primaril...,**business**,politics
1,politics,crisis ahead in social sciences a national b...,tech,politics,"business, politics",**tech/science**,politics
2,tech,football manager scores big time for the past ...,entertainment,sport,"sports,football,gaming,business (for the compa...",**tech**,tech
3,politics,uk will stand firm on eu rebate britain s £3b...,politics,politics,politics,**business**,politics
4,entertainment,greer attacks bully big brother germaine gre...,entertainment,entertainment,entertainment,**entertainment**,entertainment
...,...,...,...,...,...,...,...
495,business,singapore growth at 8.1% in 2004 singapore s e...,business,business,business,**business**,business
496,business,mixed reaction to man utd offer shares in manc...,business,business,"business,sport\n\nthe text is primarily about ...",**business**,business
497,sport,mourinho plots impressive course chelsea s win...,sport,sport,"business,sport\n\nthe text is primarily about ...",**business**,sports
498,tech,supercomputer breaks speed record the us is po...,tech,tech,technology\n\nconfidence: 95%,**tech**,tech


In [40]:
df.isnull().sum()

category                                 0
text                                     0
predicted_category_Llama3-8b-8192        0
predicted_category_Llama3-70b-8192       0
predicted_category_Mixtral-8x7b-32768    0
predicted_category_Gemma-7b-It           0
predicted_category_gpt-3.5-turbo         0
dtype: int64

In [41]:
df.columns = ['target','text','Llama3-8b','Llama3-70b','Mixtral-8x7b','Gemma-7b','gpt-3.5-turbo']
df

,target,text,Llama3-8b,Llama3-70b,Mixtral-8x7b,Gemma-7b,gpt-3.5-turbo
0,politics,nhs waiting time target is cut hospital waitin...,politics,others,politics_x000D_\n_x000D_\nthe text is primaril...,**business**,politics
1,politics,crisis ahead in social sciences a national b...,tech,politics,"business, politics",**tech/science**,politics
2,tech,football manager scores big time for the past ...,entertainment,sport,"sports,football,gaming,business (for the compa...",**tech**,tech
3,politics,uk will stand firm on eu rebate britain s £3b...,politics,politics,politics,**business**,politics
4,entertainment,greer attacks bully big brother germaine gre...,entertainment,entertainment,entertainment,**entertainment**,entertainment
...,...,...,...,...,...,...,...
495,business,singapore growth at 8.1% in 2004 singapore s e...,business,business,business,**business**,business
496,business,mixed reaction to man utd offer shares in manc...,business,business,"business,sport\n\nthe text is primarily about ...",**business**,business
497,sport,mourinho plots impressive course chelsea s win...,sport,sport,"business,sport\n\nthe text is primarily about ...",**business**,sports
498,tech,supercomputer breaks speed record the us is po...,tech,tech,technology\n\nconfidence: 95%,**tech**,tech


In [42]:
import re

def extract_alphabetic_characters(text: str) -> str:
    # Find all alphabetic sequences
    matches = re.findall(r'[a-zA-Z]+', text)
    # Join them back into a single string
    return ''.join(matches)

# Apply the function to the 'text' column
df['Gemma-7b'] = df['Gemma-7b'].apply(extract_alphabetic_characters)
df['Gemma-7b'] = df['Gemma-7b'].replace({'techscience':'tech','technopoliticsyesentertainmentnobusiness':'others','techpoliticsentertainment':'others',
                                                                        'predictiongames':'sport', 'techbusinessentertainmentsports':'sport'})
df['Gemma-7b'].value_counts()

business         180
sport            103
entertainment     80
politics          68
tech              67
others             2
Name: Gemma-7b, dtype: int64

In [43]:
df['Llama3-8b'] = df['Llama3-8b'].replace({'**tech**':'tech'})
df['Llama3-70b'] = df['Llama3-70b'].replace({'the final prediction is: tech':'tech','the final prediction is: others':'others','the predicted category is: tech':'tech'})
df['gpt-3.5-turbo'] = df['gpt-3.5-turbo'].replace({"""the text can be classified as "politics".""" : "politics",'sports':'sport'})


df.iloc[:,2].value_counts()

politics         128
sport            107
business          95
entertainment     89
tech              81
Name: Llama3-8b, dtype: int64

In [44]:
df.iloc[:,3].value_counts()

sport            112
business         111
politics         105
entertainment     91
tech              77
others             4
Name: Llama3-70b, dtype: int64

In [45]:
df.iloc[:,5].value_counts()

business         180
sport            103
entertainment     80
politics          68
tech              67
others             2
Name: Gemma-7b, dtype: int64

In [46]:
df.iloc[:,6].value_counts()

politics         118
business         111
sport            106
tech              82
entertainment     82
others             1
Name: gpt-3.5-turbo, dtype: int64

In [47]:
import re

def substitute_words(text: str) -> str:
    # Define the words to find and substitute
    words_to_substitute = {"tech", "politics", "entertainment", "business", "sport", "others"}
    
    # Regex pattern to match any of the words as whole words
    pattern = r'\b(?:{})\b'.format('|'.join(words_to_substitute))
    
    # Substitute the matched words with themselves
    text = re.sub(pattern, lambda match: match.group(0), text)
    
    # Preprocess the text to remove additional information
    text = re.sub(r'(the final prediction is:|confidence:).*', '', text, flags=re.IGNORECASE)
    
    return text.strip()

In [48]:
df['Mixtral-8x7b'] = df['Mixtral-8x7b'].apply(substitute_words) ## so so bad

In [49]:
df.to_excel("data/news_category_classification/Final.xlsx", index=False)
